# The Empathetic Code Reviewer ✨

> Transforming Critical Feedback into Constructive Growth.

This project is an AI-powered tool built to solve a common problem in software development: harsh, impersonal code reviews. It acts as an intelligent bridge, taking any Python code snippet, automatically identifying areas for improvement, and then reframing the critical feedback into a supportive, educational, and highly detailed report.

---
## Key Features

* **🤖 Automated Code Review:** Simply provide a code snippet, and the AI will act as a senior developer to identify potential issues on its own.
* **⚙️ Automatic Formatting:** Before the review, the script automatically formats the input code to comply with PEP 8 standards using the `autopep8` library.
* **🧠 Two-Step AI Analysis:**
    1.  A "Critical Reviewer" persona finds and lists potential issues.
    2.  An "Empathetic Mentor" persona takes those issues and generates a detailed, constructive report.
* **📚 Detailed & Specific Feedback:** For each issue found, the final report provides a positive rephrasing, a clear explanation of the "why" (the underlying principle), and a specific code example showing the suggested improvement.

---
## 🚀 How to Use

This project is designed to be run in a single Google Colab notebook.

### 1. Prerequisites
* A Google Account to use Google Colab.
* A Groq Cloud account to get a free API key.

### 2. Setup
1.  **Get Groq API Key:** Go to the [GroqCloud Console](https://console.groq.com/keys) and create a new, free API key. Copy it.
2.  **Open Google Colab:** Go to [colab.research.google.com](https://colab.research.google.com) and create a new notebook.
3.  **Add API Key to Secrets:**
    * In your Colab notebook, click the **key icon (🔑)** on the left sidebar.
    * Create a new secret named `GROQ_API_KEY`.
    * Paste your Groq API key into the "Value" field and save it.
4.  **Run the Code:** Copy the complete Python script for the project into a single cell in your notebook and run it.

### 3. Provide Input
1.  After running the cell, it will prompt you to enter your code.
2.  Paste your Python code snippet.
3.  On a new, empty line, type `ENDCODE` and press Enter to start the analysis.
4.  The formatted code and the final empathetic review will be displayed in the output.

---
## 🛠️ Technologies Used

* **Python 3**
* **Groq API** (Leveraging Llama 3 models)
* **autopep8** (For code formatting)
* **Google Colab** (As the development environment)

---
## 🙏 Acknowledgments

A heartfelt thank you to the entire team at **Darwix.ai**. Your dedication, innovation, and commitment to excellence are truly inspiring and have been a driving force behind this project. Thank you for all that you do!

In [45]:
# 1. Install the Groq library
!pip install -q groq
print("✅ Groq library installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 5.3 MB/s eta 0:00:00
✅ Groq library installed.


In [55]:
# 1. Install necessary libraries
!pip install -q groq autopep8
print("✅ Libraries (groq, autopep8) installed.")

✅ Libraries (groq, autopep8) installed.


In [56]:
# 2. Import libraries
from groq import Groq
from google.colab import userdata
from IPython.display import display, Markdown, clear_output
import json
import autopep8

In [57]:
# 3. Configure the Groq client
groq_client = None
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    groq_client = Groq(api_key=groq_api_key)
    print("✨ Groq client configured successfully!")
except Exception as e:
    print("🔴 ERROR: Could not configure Groq client. Please check your 'GROQ_API_KEY' secret.")

✨ Groq client configured successfully!


In [58]:
# --- Helper function for multi-line input ---
def get_multiline_input(end_keyword):
    """Reads multiple lines of input from the user."""
    print(f"👇 Paste your code below. Type {end_keyword} on a new line when you are finished.")
    lines = []
    while True:
        try:
            line = input()
            if line.strip().upper() == end_keyword:
                break
            lines.append(line)
        except EOFError:
            break
    return "\n".join(lines)

In [59]:
# --- AI Logic ---
def run_automated_review(user_code):
    if not groq_client:
        return "🔴 Groq client not configured.", None

    # --- Auto-format the user's code ---
    print("\n⚙️ Auto-indenting and formatting your code...")
    formatted_code = autopep8.fix_code(user_code)
    print("✅ Code formatting complete.")

    # --- STEP 1: AI acts as a CRITICAL reviewer ---
    print("\n🤖 Step 1: Generating critical review comments...")
    critical_reviewer_prompt = f"""
    You are a meticulous senior developer. Review the following Python code. Identify 3-4 areas for improvement focusing on efficiency, naming, redundancy, and logic.
    Your output MUST be a JSON object with a single key "comments" that contains an array of strings. Each string is a single, concise, critical comment.
    Code to review:
    ```python
    {formatted_code}
    ```
    """
    try:
        critical_response = groq_client.chat.completions.create(
            messages=[{"role": "user", "content": critical_reviewer_prompt}],
            model="llama3-8b-8192", temperature=0.1, response_format={"type": "json_object"},
        )
        harsh_comments = json.loads(critical_response.choices[0].message.content).get("comments", [])
        if not harsh_comments:
             return "🔴 AI failed to generate review comments.", None
        print("✅ Critical comments generated.")
    except Exception as e:
        return f"🔴 An error occurred during Step 1 (Critical Review): {e}", formatted_code

    # --- STEP 2: AI acts as an EMPATHETIC mentor with CORRECTED, STRICTER instructions ---
    print("\n🤖 Step 2: Generating the empathetic report...")
    empathetic_mentor_prompt = f"""
    You are an expert senior software developer and a patient, empathetic mentor. Your mission is to transform the following blunt, critical code review comments into a supportive, educational, and constructive report.

    **CRITICAL INSTRUCTIONS:**
    1.  You must process **every single comment** from the list provided.
    2.  For each comment, generate a distinct Markdown section starting with `---` and `### Analysis of Comment: "[Original Comment]"`.
    3.  Each section must contain three bullet points: `**Positive Rephrasing:**`, `**The 'Why':**`, and `**Suggested Improvement:**`.
    4.  The `**Suggested Improvement**` code block is crucial. It **must only show the specific fix for that one comment**. Do not combine suggestions. For example:
        - If a comment is about a variable name, the code block should only show the name changed.
        - If a comment is about a boolean check, the code block should only show that part fixed.
    5.  After processing all comments, add a final `## Overall Summary & Encouragement` section.

    Here is the formatted code snippet that was reviewed:
    ```python
    {formatted_code}
    ```

    Here are the critical comments to transform:
    {json.dumps(harsh_comments)}
    """
    try:
        empathetic_response = groq_client.chat.completions.create(
            messages=[{"role": "user", "content": empathetic_mentor_prompt}],
            model="llama3-70b-8192",
        )
        final_report = empathetic_response.choices[0].message.content
        return final_report, formatted_code
    except Exception as e:
        return f"🔴 An error occurred during Step 2 (Empathetic Review): {e}", formatted_code


In [63]:
# --- Main Execution Block ---
if groq_client:
    user_code_snippet = get_multiline_input("ENDCODE")
    if user_code_snippet.strip():
        report, final_code = run_automated_review(user_code_snippet)
        clear_output()
        if final_code:
            display(Markdown("---"))
            display(Markdown("### ✨ Auto-Formatted Code"))
            display(Markdown(f"```python\n{final_code}\n```"))
        if report:
            display(Markdown("---"))
            display(Markdown("### 📝 Empathetic Code Review"))
            display(Markdown(report))
    else:
        print("\n🔴 No code entered. Analysis cancelled.")

---

### ✨ Auto-Formatted Code

```python
# Output: nohtyp
original_string = "python" reversed_string = "" for char in original_string:     reversed_string = char + reversed_string print(reversed_string)

```

---

### 📝 Empathetic Code Review

🔴 An error occurred during Step 2 (Empathetic Review): Error code: 503 - {'error': {'message': 'Service unavailable. Visit https://groqstatus.com/ to see if there is an active incident.', 'type': 'internal_server_error'}}